In [2]:
import json

with open('pa-data-common.json', 'r') as f:
    data = json.load(f)
    
import openpyxl
from os import listdir
from os.path import isfile, join
mypath = '../vpscrape/getvirons/xlfiles/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

from plot_util import *

def letters(s):
    valids = []
    for character in s:
        if character.isalpha():
            valids.append(character)
    return ''.join(valids)

In [3]:
mypath = '../vpscrape/getvirons/xlfiles/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
idpath = '../vpscrape/getvirons/xlfiles_prechain/'
idpafiles = [f for f in listdir(idpath) if isfile(join(idpath, f))]
id_pa = {}
for f in idpafiles:
    try:
        if 'full' in f: continue
        v = f.split('.xlsx')[0]
        xldir = idpath + f
        wb = openpyxl.load_workbook(xldir)
        name = wb.sheetnames[0]
        sh = wb[name]
        r = sh[2]
        closest_pa = r[0].value
        id_pa[v] = closest_pa
    except Exception as e:
        print(v)

with open('id_pa.txt', 'w') as f:
    for i in id_pa:
        f.write(i + ' ' + str(id_pa[i])+ '\n')

data['close_aas'] = {}
for f in onlyfiles:
    if 'full' not in f or '.xlsx' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    pa = id_pa[v]
    pa_sh = f"{v}_pa_{pa}"
    resdict = []
    sh = wb[pa_sh]
    vitems = []
    for row in sh.iter_rows():
            vitems.append([cell.value for cell in row])
    aas = []
    ds_aas = {}
    gp_aas = {}
    for j in range(2, len(vitems)):
        for i in range(len(vitems[0])):
            if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                d = vitems[j][i]
                ds_aas[d] = vitems[j][i:i+4]
                if j == 2:
                    gp_aas[d] = ds_aas[d]
                if d < 5:
                    aas.append(letters(vitems[j][i+2]))
                    if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
    resdict.extend(aas)
    closest_gp_aa = letters(gp_aas[min(gp_aas.keys())][2])
    other_gp_aa = letters(gp_aas[min(gp_aas.keys())][3])
    closest_aa = letters(ds_aas[min(ds_aas.keys())][2])
    other_aa = letters(ds_aas[min(ds_aas.keys())][3])
    data['data'][v]['closest_gp_aa'] = closest_gp_aa
    data['data'][v]['other_gp_aa'] = other_gp_aa
    data['data'][v]['closest_aa'] = closest_aa
    data['data'][v]['other_aa'] = other_aa
    cres = countlist(resdict)
    data['close_aas'][v] = cres
    try:
        data['data'][v]['common_gauge_aa'] = max(cres, key=cres.get)
    except Exception as e:
        print(v)

for f in onlyfiles:
    if 'full' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    resdict = []
    for name in wb.sheetnames:
            sh = wb[name]
            vitems = []
            for row in sh.iter_rows():
                    vitems.append([cell.value for cell in row])
            aas = []
            cnt = 0
            for i in range(len(vitems[0])):
                for j in range(2, len(vitems)):
                    if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                        d = vitems[j][i]
                        if d < 5:
                            aas.append(letters(vitems[j][i+2]))
                            if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
            resdict.extend(aas)
    cres = countlist(resdict)
    data['close_aas'][v] = cres
    data['data'][v]['common_gauge_aa'] = max(cres, key=cres.get)

del data['closest_aas']

del data['gauge_points']

In [14]:
#was pa-data-close.json
with open('pa-data-final.json', 'w') as f:
    json_str = json.dumps(data, indent=4)
    f.write(json_str)

In [34]:
with open('pa-data-dclosest.json', 'w') as f:
    json_str = json.dumps(data, indent=4)
    f.write(json_str)

In [17]:
with open('pa-data-gp-aas.json', 'w') as f:
    json_str = json.dumps(data, indent=4)
    f.write(json_str)